In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, layers, Input
from keras.layers import Dense, InputLayer, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Model, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.layers import Conv1D, MaxPool1D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping

2024-03-01 19:15:11.491123: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-01 19:15:11.676704: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 19:15:12.618275: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 19:15:12.618309: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 19:15:12.621389: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [ ]:
from pathlib import Path
from tensorflow.keras.utils import image_dataset_from_directory
# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 10



test_dir ='/kaggle/input/train-test/split/test'

# test_ds = image_dataset_from_directory(
# folder_test,
# labels = 'inferred',
# seed=123,
# batch_size=batch_size)


train_dir='/kaggle/input/train-test/split/train'

# train_ds, val_ds = image_dataset_from_directory(
# folder_train,
# subset = 'both',
# validation_split = 0.2,
# labels = 'inferred',
# seed=123,
# batch_size=batch_size)

In [ ]:
# Data Generators
train_gen = ImageDataGenerator(
    rescale=1./255.,
    validation_split=0.2,  # Splitting data for validation
)
test_gen = ImageDataGenerator(rescale=1./255.)
batchSize = 32
imageShape = (224, 224)

In [ ]:
# Preparing the datasets
train_ds = train_gen.flow_from_directory(
    train_dir,
    target_size=imageShape,
    batch_size=batchSize,
    subset='training',
    class_mode='categorical'  # Using 'categorical' for multi-class classification
)
val_ds = train_gen.flow_from_directory(
    train_dir,
    target_size=imageShape,
    batch_size=batchSize,
    subset='validation',
    class_mode='categorical'
)
test_ds = test_gen.flow_from_directory(
    test_dir,
    target_size=imageShape,
    batch_size=batchSize,
    class_mode='categorical'
)

In [ ]:
# Model Definition
model = models.Sequential([
    Input((224, 224, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')  # Adjust this based on your number of classes
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['recall'])  # It's common to monitor accuracy for classification tasks
# Make sure to import EarlyStopping from tensorflow.keras.callbacks

In [ ]:
# Early Stopping Callback
early_stopping = EarlyStopping(monitor="val_loss", patience=5)
# Model Training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
#     steps_per_epoch=len(train_ds),
#     validation_steps=len(val_ds),
    callbacks=[early_stopping]  # Using the early stopping callback
)

In [ ]:
model.save('Maha\'s notebook_with_input.h5')

In [ ]:
model.get_config()